<a href="https://colab.research.google.com/github/guanganalytics/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,130 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,009 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,305 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://archi

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [70]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""
SELECT  year(date) yr, round(AVG(price),2) avgprice
from home_sales
where bedrooms = 4
group by yr
order by yr asc
""").show()


+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+
| id|date|date_built|price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+
+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+

+----+---------+
|  yr| avgprice|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [67]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built, round(AVG(price),2) avgprice
from home_sales
where bedrooms = 3 and bathrooms = 3
group by date_built
order by date_built asc
""").show()


+----------+---------+
|date_built| avgprice|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [71]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT date_built, round(AVG(price),2) avgprice
from home_sales
where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living>= 2000
group by date_built
order by date_built asc
""").show()


+----------+---------+
|date_built| avgprice|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [77]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select id, view
from home_sales
where price>= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|                  id|view|
+--------------------+----+
|f8a53099-ba1c-47d...|  76|
|7530a2d8-1ae3-451...|  23|
|43de979c-0bf0-4c9...|   0|
|e0726d4d-d595-407...|   4|
|131492a1-72e2-4a8...|   6|
|f1e4cef7-d151-439...|   7|
|ea620c7b-c2f7-4c6...|  26|
|f233cb41-6f33-4b0...|  34|
|164f451a-413b-469...|  23|
|2dc662fe-57b8-44b...|  48|
|a16c1fd3-50c3-4e7...|  16|
|d715f295-2fbf-4e9...|  10|
|a18515a2-86f3-46b...|  24|
|15a5830f-1529-4c5...|  91|
|0ffe23e3-9d0f-442...|  49|
|209c1286-0834-463...|  37|
|bb2ff269-f08f-4e8...|  23|
|77a2d239-67a3-43b...|  25|
|141ef834-60cf-42e...|  23|
|34c31a34-220d-469...|   1|
+--------------------+----+
only showing top 20 rows

--- 0.1309816837310791 seconds ---


In [78]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [79]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [80]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""select id, view
from home_sales
where price>= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|                  id|view|
+--------------------+----+
|f8a53099-ba1c-47d...|  76|
|7530a2d8-1ae3-451...|  23|
|43de979c-0bf0-4c9...|   0|
|e0726d4d-d595-407...|   4|
|131492a1-72e2-4a8...|   6|
|f1e4cef7-d151-439...|   7|
|ea620c7b-c2f7-4c6...|  26|
|f233cb41-6f33-4b0...|  34|
|164f451a-413b-469...|  23|
|2dc662fe-57b8-44b...|  48|
|a16c1fd3-50c3-4e7...|  16|
|d715f295-2fbf-4e9...|  10|
|a18515a2-86f3-46b...|  24|
|15a5830f-1529-4c5...|  91|
|0ffe23e3-9d0f-442...|  49|
|209c1286-0834-463...|  37|
|bb2ff269-f08f-4e8...|  23|
|77a2d239-67a3-43b...|  25|
|141ef834-60cf-42e...|  23|
|34c31a34-220d-469...|   1|
+--------------------+----+
only showing top 20 rows

--- 0.11747431755065918 seconds ---


In [82]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [84]:
# 11. Read the parquet formatted data.
p_df_p = spark.read.parquet('home_sales_partitioned')
p_df_p.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|     2021|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|     2020|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|     2019|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|     2019|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|     2021|      2015|


In [86]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_p')

In [87]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""select id, view
from p_home_sales_p
where price>= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|                  id|view|
+--------------------+----+
|d715f295-2fbf-4e9...|  10|
|a18515a2-86f3-46b...|  24|
|34c31a34-220d-469...|   1|
|be0ccb95-415d-411...|   4|
|00fc996f-508c-430...|  39|
|3d5545f8-bd3b-476...|  90|
|ec6d357c-2435-43e...|  50|
|c2be38fb-814a-403...|   6|
|5112f184-7a3f-4f7...|   0|
|a4a1a614-ad3c-4f8...|  26|
|1ebb9526-ad9f-4eb...|  17|
|40ce4e9a-1459-450...|  10|
|ac4be07f-8731-4aa...|  58|
|c4b80d59-5ff2-4c5...|  82|
|38c843f4-ccd8-41f...|   2|
|fc49b8f7-c325-45d...|  44|
|d3d65bfe-ae9f-4eb...|  45|
|760b227e-56b1-450...|  88|
|d098c5f1-8ae4-49a...|  49|
|3397c4c3-25fe-46c...|  56|
+--------------------+----+
only showing top 20 rows

--- 0.5185930728912354 seconds ---


In [88]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [89]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False